In [1]:
import numpy as np
import pandas as pd
import os
import re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')

In [2]:
def get_dir_list(dir_path):
    path_names = []
    dirs = os.listdir(dir_path)
    for file in dirs: 
        path_names.append(dir_path + "/" + file)

    return path_names   

In [3]:
def get_file_info(file_path_name, session_name):
    # takes in a file path, finds the animal_number and session and saves each accordingly
    
    path_split = file_path_name.split('/')
    file_split = path_split[-1].split('.')
    animal_number, session = file_split[0].split('_')

    return animal_number, session

In [4]:
def create_animal_df(file_path_name, animal_number, session):
    # takes in a file path, creates data frame, adds animal number and session columns
    
    data = pd.read_table(file_path_name)
    data = pd.DataFrame(data = data)
    
    data['animal_number'] = [animal_number]*data.shape[0]
    data['session'] = [session]*data.shape[0]
    
    #Determine closest time stamp of each annotation and add as column to df
    data['Begin Time (s)_1000'] = data['Begin Time (s)']*1000
    time_lambda = lambda a: (a // 22.5)*22.5
    data['time_stamp'] = time_lambda(data['Begin Time (s)_1000'])
    
    data = data[['animal_number', 'session', 'time_stamp', 'Annotation']]

    return data

In [5]:
def create_annot_df(dir_path, session_name):
    annot_df = pd.DataFrame()
    
    path_names = get_dir_list(dir_path)
    for path in path_names:
        animal_number, session = get_file_info(path, session_name)
        animal_df = create_animal_df(path, animal_number, session)
        annot_df = annot_df.append(animal_df)
    
    #save as csv
    annot_df.to_csv(str('annot_df_' + str(dir_path.split('/')[-1]) + '.csv'))
    
    return annot_df

In [8]:
dir_path = "C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/Annotation_tables/3x_CPA_pair_tables"

In [9]:
annot_df = create_annot_df(dir_path, 'CPApair')
print(annot_df.shape)

(459, 4)
